# __Predicting NTL using DTL__

Runs CNN to predict nighttime lights using daytime images

# __Parameters__

In [ ]:
# Test
# Test2
#PARAM_NAME = "Nbands3_nNtlBins3_minNTLbinCount16861"
PARAM_NAME = "Nbands3_nNtlBins3_minNTLbinCount100"
YEAR = 2014

## **Filepaths and Libraries**

In [ ]:
# Set seeds. Note that using a GPU can still introduce randomness.
# (also not taking into account tensorflow randomness)
from numpy.random import seed
seed(42)

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#drive.flush_and_unmount()

In [ ]:
# Filepaths
import os, datetime

CNN_DIR = os.path.join('/content/drive', 'My Drive', 'World Bank', 'IEs', 'Pakistan Poverty Estimation', 'Data', 'CNN', PARAM_NAME)

CNN_FILENAME = os.path.join(CNN_DIR, f'script_CNN_{str(YEAR)}.h5')
CNN_CONT_FILENAME = os.path.join(CNN_DIR, f'script_CNN_cont_{str(YEAR)}.h5') # continuous

CNN_PARAMS_FILENAME = os.path.join(CNN_DIR, 'CNN_parameters.json')
NTL_FILENAME = os.path.join(CNN_DIR, f'ntl_{str(YEAR)}.npy')
NTL_CONT_FILENAME = os.path.join(CNN_DIR, f'ntl_continuous_{str(YEAR)}.npy')
DTL_FILENAME = os.path.join(CNN_DIR, f'dtl_{str(YEAR)}.npy')
PREDICTION_FILENAME = os.path.join(CNN_DIR, f'cnn_predictions_truth_values_{str(YEAR)}.csv')
PREDICTION_FILENAME_CONT = os.path.join(CNN_DIR, f'cnn_predictions_continuous_truth_values_{str(YEAR)}.csv')

In [ ]:
import os, datetime
import numpy as np
import pandas as pd
import json

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix

from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import preprocess_input

import logging, os 
import random
import tensorflow as tf

## **Functions**

In [ ]:
def transform_target(gdf, orig_target_name, n_bins):
    '''
    Creates log NTL variable and bins into 5 classes using k-means clutering.
    '''
    # Perform log(x+1) for defined domain
    transformed_target_name = f'log_{orig_target_name}'
    gdf[transformed_target_name] = np.log(gdf[orig_target_name] + 1)
    # Bin target
    target = gdf[transformed_target_name].to_numpy().reshape(-1,1)
    discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='kmeans')
    gdf[FINAL_TARGET_NAME] = discretizer.fit_transform(target)

def normalize(X):
    '''
    Normalizes features.
    '''
    return X.astype('float32') / 255.0

def define_model_imagenet(height, width, channels, num_classes):
    '''
    Defines and compiles CNN model.
    
    Inputs:
        height, width, channels, num_classes (int)
    Returns:
        model (keras.Model object)
    '''

    # https://medium.com/abraia/first-steps-with-transfer-learning-for-custom-image-classification-with-keras-b941601fcad5
    # https://towardsdatascience.com/cnn-transfer-learning-fine-tuning-9f3e7c5806b2

    #### Base model
    input_shape = (height, width, channels)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    for layer in base_model.layers:
        layer.trainable = False

    #### Model Customization
    # We take the last layer of our the model and add it to our classifier
    last = base_model.layers[-1].output
    x = Flatten()(last)
    x = Dense(100, activation='relu', name='fc1')(x)
    x = Dropout(0.3)(x)
    x = Dense(num_classes, activation='softmax', name='predictions')(x)
    model = Model(base_model.input, x)
    # We compile the model
    model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    return model

def define_model_imagenet_cont(height, width, channels, num_classes):
    '''
    Defines and compiles CNN model [continuous].
    
    Inputs:
        height, width, channels, num_classes (int)
    Returns:
        model (keras.Model object)
    '''

    # https://medium.com/abraia/first-steps-with-transfer-learning-for-custom-image-classification-with-keras-b941601fcad5
    # https://towardsdatascience.com/cnn-transfer-learning-fine-tuning-9f3e7c5806b2

    #### Base model
    input_shape = (height, width, channels)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    for layer in base_model.layers:
        layer.trainable = False

    #### Model Customization
    # We take the last layer of our the model and add it to our classifier
    last = base_model.layers[-1].output
    x = Flatten()(last)
    x = Dense(100, activation='relu', name='fc1')(x)
    x = Dropout(0.3)(x)
    #x = Dense(num_classes, activation='softmax', name='predictions')(x)
    x = Dense(1, kernel_initializer='normal')(x)

    model = Model(base_model.input, x)
    # We compile the model
    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

    return model


def evaluate_model(model, trainX, trainY, testX, testY, CNN_FILENAME):
    '''
    Fits model, evaluates model, saves best model over epochs and cross-validations.
    
    Inputs:
        model (CNN model) keras.Model object
        trainX, trainY (numpy.ndarray) 4D array of DTL features and 2D array of targets for training
        testX, testY (numpy.ndarray) 4D array of DTL features and 2D array of targets for testing
        current_kfold (int) iteration in kfold cross-val, default=None for no cross-val
        display_metrics (bool) Default=False
    Returns:
        None
    # https://towardsdatascience.com/step-by-step-guide-to-using-pretrained-models-in-keras-c9097b647b29
    '''

    # Use early stopping to help with overfitting
    es = EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose=False)

    # Save best model based on accuracy
    mc = ModelCheckpoint(CNN_FILENAME, monitor='val_loss', mode='min', 
                         verbose=True, save_best_only=True)

    # Fit model
    model.fit(trainX, trainY, 
            epochs=100, 
            batch_size=500, 
            validation_data=(testX, testY), 
            callbacks=[es, mc], 
            verbose=False)

    # Show accuracy
    loss, accuracy = model.evaluate(testX, testY, verbose=False)
    print(f'                              Accuracy: {accuracy}')

    #return model
        

def evaluate_with_crossval(model, dataX, dataY, k=2):
    '''
    Performs evaulation with K-fold cross validation.
    
    Inputs:
        model (keras.Model object)
        dataX, dataY (numpy.ndarray) 4D array of DTL features and 2D array of targets 
                                     for training
        k (int)
    Returns:
        None
    '''
    # Define k-fold cross-val
    kfold = KFold(k, shuffle=True, random_state=1)
    # Loop through folds
    count = 1
    for train_idx, test_idx in kfold.split(dataX):
        print(f'{datetime.datetime.now()}    --- Current K-fold: {count} ---')
        # Select subsets for training and testing
        trainX, trainY, testX, testY = dataX[train_idx], dataY[train_idx], \
                                       dataX[test_idx], dataY[test_idx]
        # Pass to evaluate_model function
        evaluate_model(model, trainX, trainY, testX, testY)
        count += 1

def display_eval_metrics(model, testX, testY, n_ntl_bins):
    '''
    Displays evaluation metrics for a given trained model.
    '''
    # Get predictions
    predY = model.predict(testX)
    predY = np.argmax(predY, axis = 1)
    testY_bins = np.argmax(testY, axis = 1)
    # Generate classification report
    classes = ['Radiance Level %01d' %i for i in range(1,n_ntl_bins+1)]
    print(classification_report(testY_bins, predY, target_names=classes))

## **Load Parameters**

In [ ]:
with open(CNN_PARAMS_FILENAME, 'r') as fp:
    cnn_param_dict = json.load(fp)

N_bands = cnn_param_dict['N_bands']
n_ntl_bins = cnn_param_dict['n_ntl_bins']
image_height = cnn_param_dict['image_height']
image_width = cnn_param_dict['image_width']
bands = cnn_param_dict['bands']
min_ntl_bin_count = cnn_param_dict['bands']

## **Load and Prep Data**

In [ ]:
# Load Data
NTL = np.load(NTL_FILENAME)
DTL = np.load(DTL_FILENAME)

# SPLIT DATA INTO TRAINING AND TESTING
trainX, testX, raw_trainY, raw_testY = train_test_split(DTL, NTL, 
                                                        test_size=0.2)


# PREP TRAINING AND TESTING DATA
trainY = to_categorical(raw_trainY)
testY = to_categorical(raw_testY)


#print(np.unique(NTL, return_counts=True))

#print(np.unique(raw_trainY, return_counts=True))
#print(np.unique(raw_testY, return_counts=True))

#print(np.unique(trainY, return_counts=True))
#print(np.unique(testY, return_counts=True))

# PREP PIXELS IN FEATURES
trainX, testX = normalize(trainX), normalize(testX)

In [ ]:
# Prep Continuous X and Ys
NTL = np.load(NTL_CONT_FILENAME)

NTL_LOG = np.log(NTL + 1)

trainX_cont, testX_cont, raw_trainY_cont, raw_testY_cont = train_test_split(DTL, NTL_LOG, 
                                                                            test_size=0.2)

trainX_cont, testX_cont = normalize(trainX_cont), normalize(testX_cont)

In [ ]:
model_cont = define_model_imagenet_cont(image_height, image_width, N_bands, n_ntl_bins)

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
evaluate_model(model_cont, trainX_cont, raw_trainY_cont, testX_cont, raw_testY_cont, CNN_CONT_FILENAME)


Epoch 00001: val_loss improved from inf to 1.61814, saving model to /content/drive/My Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/CNN/Nbands3_nNtlBins3_minNTLbinCount100/script_CNN_2014.h5

Epoch 00002: val_loss improved from 1.61814 to 1.24220, saving model to /content/drive/My Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/CNN/Nbands3_nNtlBins3_minNTLbinCount100/script_CNN_2014.h5

Epoch 00003: val_loss improved from 1.24220 to 1.02511, saving model to /content/drive/My Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/CNN/Nbands3_nNtlBins3_minNTLbinCount100/script_CNN_2014.h5

Epoch 00004: val_loss improved from 1.02511 to 0.94041, saving model to /content/drive/My Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/CNN/Nbands3_nNtlBins3_minNTLbinCount100/script_CNN_2014.h5

Epoch 00005: val_loss did not improve from 0.94041

Epoch 00006: val_loss did not improve from 0.94041

Epoch 00007: val_loss did not improve from 0.94041

Epoch 00008: val_loss did not i

In [ ]:
best_model = load_model(CNN_CONT_FILENAME)

### Save Dataframe of Predicted Values

# Predict Values
predY = best_model.predict(testX_cont)
#predY = np.argmax(predY, axis = 1)
#testY_bins = np.argmax(testY, axis = 1)

# Make Dataframe
results_df = pd.DataFrame({'predY': predY[:,0], 'testY': raw_testY_cont})

# Save Dataframe
results_df.to_csv(PREDICTION_FILENAME_CONT, index=False)  

In [ ]:
#results_df.head()
#raw_testY_cont
#predY
results_df.head()

,predY,testY
0,0.919813,1.482614
1,0.872260,0.546836
2,1.130655,0.386482
3,0.805316,0.117846
4,1.092091,0.454955


## **Run Model**

In [ ]:
model = define_model_imagenet(image_height, image_width, N_bands, n_ntl_bins)

In [ ]:
evaluate_model(model, trainX, trainY, testX, testY, CNN_FILENAME)


Epoch 00001: val_loss improved from inf to 1.23411, saving model to /content/drive/My Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/CNN/Nbands3_nNtlBins3_minNTLbinCount100/script_CNN_2014.h5

Epoch 00002: val_loss improved from 1.23411 to 1.12425, saving model to /content/drive/My Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/CNN/Nbands3_nNtlBins3_minNTLbinCount100/script_CNN_2014.h5

Epoch 00003: val_loss improved from 1.12425 to 1.06902, saving model to /content/drive/My Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/CNN/Nbands3_nNtlBins3_minNTLbinCount100/script_CNN_2014.h5

Epoch 00004: val_loss improved from 1.06902 to 1.04433, saving model to /content/drive/My Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/CNN/Nbands3_nNtlBins3_minNTLbinCount100/script_CNN_2014.h5

Epoch 00005: val_loss improved from 1.04433 to 1.03642, saving model to /content/drive/My Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/CNN/Nbands3_nNtlBins3_minNTLbinCount100/scr

In [ ]:
# DISPLAY IN-DEPTH EVALUTAION METRICS
best_model = load_model(CNN_FILENAME)
display_eval_metrics(model, testX, testY, n_ntl_bins)

                  precision    recall  f1-score   support

Radiance Level 1       0.65      0.59      0.62        22
Radiance Level 2       0.43      0.38      0.40        16
Radiance Level 3       0.62      0.73      0.67        22

        accuracy                           0.58        60
       macro avg       0.56      0.56      0.56        60
    weighted avg       0.58      0.58      0.58        60



In [ ]:
### Save Dataframe of Predicted Values

# Predict Values
predY = best_model.predict(testX) # model.predict(testX)
predY = np.argmax(predY, axis = 1)
testY_bins = np.argmax(testY, axis = 1)

# Make Dataframe
results_df = pd.DataFrame({'predY': predY, 'testY': testY_bins})

# Save Dataframe
results_df.to_csv(PREDICTION_FILENAME, index=False)  